In [2]:
import snowflake.connector
import pandas as pd

from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

CONEXÃO

In [3]:
from dotenv import load_dotenv
import os

load_dotenv('config.env')

user = os.getenv('user')
password = os.getenv('password')
account = os.getenv('account')
warehouse = os.getenv('warehouse')
database = os.getenv('database')
schema = os.getenv('schema')

In [4]:
conn = snowflake.connector.connect(
    user=user
    , password=password
    , account=account 
    , warehouse=warehouse
    , database=database   
    , schema=schema       
)

In [3]:
cursor = conn.cursor()

In [ ]:
cursor.close()
conn.close()

CARREGAR OS DADOS

In [4]:
query = """SELECT 
            NM_CLIENTE
            , NOME_PRODUTO
            , ANO
            , MES
            , DATA_REF
            , QUANTIDADE
            , REGIAO
        from agg_sales_store """

cursor.execute(query)
rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
df_cliente = pd.DataFrame(rows, columns=columns)

print(df_cliente.head(100))

           NM_CLIENTE        NOME_PRODUTO   ANO  MES    DATA_REF  QUANTIDADE  \
0   Extreme Toy Store        Front Brakes  2013   12  2013-12-01           1   
1   Extreme Toy Store   HL Bottom Bracket  2013   12  2013-12-01           2   
2   Extreme Toy Store   HL Bottom Bracket  2014    3  2014-03-01           1   
3   Extreme Toy Store   HL Mountain Pedal  2013    6  2013-06-01           2   
4   Extreme Toy Store         LL Crankset  2013    9  2013-09-01           1   
..                ...                 ...   ...  ...         ...         ...   
95  First Cycle Store  Road-250 Black, 48  2014    5  2014-05-01           3   
96  First Cycle Store  Road-250 Black, 52  2013    7  2013-07-01           1   
97  First Cycle Store  Road-250 Black, 52  2013   10  2013-10-01           1   
98  First Cycle Store  Road-250 Black, 52  2014    5  2014-05-01           4   
99  First Cycle Store  Road-250 Black, 58  2013    7  2013-07-01           1   

     REGIAO  
0   Alabama  
1   Alabama

In [5]:
df_previsao = pd.read_csv('8-previsao_demanda.csv')
print(df_previsao)

      Unnamed: 0            NOME_PRODUTO                      NM_CLIENTE  \
0              0  Mountain-200 Black, 46        Professional Cycle Store   
1              1  Mountain-200 Black, 46             Real Sporting Goods   
2              2  Mountain-200 Black, 46                Many Bikes Store   
3              3  Mountain-200 Black, 46                 Resale Services   
4              4  Mountain-200 Black, 46                 Small Bike Shop   
...          ...                     ...                             ...   
1726        1726        Road-650 Red, 44        Advanced Bike Components   
1727        1727        Road-650 Red, 44               Fitness Toy Store   
1728        1728        Road-650 Red, 44         Totes & Baskets Company   
1729        1729        Road-650 Red, 44  Sure & Reliable Sporting Goods   
1730        1730        Road-650 Red, 44                  Good Bike Shop   

      forecast_1  forecast_2  forecast_3  
0            2.5         2.8         3.0  
1

In [8]:
produtos_glove = df_previsao[df_previsao['NOME_PRODUTO'].str.contains('glove', case=False, na=False)]['NOME_PRODUTO'].unique()

print(produtos_glove)

['Half-Finger Gloves, S' 'Half-Finger Gloves, M' 'Half-Finger Gloves, L']


In [9]:
df_filtered = df_previsao[df_previsao['NOME_PRODUTO'].isin(produtos_glove)]

total_quantity = df_filtered[['forecast_1', 'forecast_2', 'forecast_3']].sum()

total_sum = round(total_quantity.sum(), 0)

final_result = total_sum * 2

print(f"Estimativa da quantidade zíperes para o novo fornecedor: {final_result}")

Estimativa da quantidade zíperes para o novo fornecedor: 1604.0
